In [2]:
from data import Course
from ouc_course_tool import *
from data.schedule import CourseTable
from tests.test_config import ConfigOfTest
from ouc_course_tool.core.search_param_handle import search_param_handle

导入包

In [3]:
test_config = ConfigOfTest('./tests/test_resource/account_local.ini')
tesseract_cmd_path = 'D:/Tools/Tesseract/tesseract.exe'

login_config = LoginConfig(test_config.get_username(), test_config.get_password(), tesseract_cmd_path)

设置 Tesseract 图片识别软件
EXAMPLE DOWNLOAD URL: https://digi.bib.uni-mannheim.de/tesseract/tesseract-ocr-w64-setup-5.3.4.20240503.exe

设置 全局变量（包括账号和密码）
设置 登录配置


In [4]:
session_id = AccountValidation(login_config).get_login_session_id()
print(session_id)

388756E29B488A5F20C85C2A4A4005EC


获取登录 ID

In [5]:
fetcher_config = FetcherConfig(session_id=session_id)
course_fetcher = CourseFetcher(config=fetcher_config)

设置 搜索配置

In [47]:
target_course = [
    # '围棋-选修',
    '概率-公共',
    '形势与政策-公共',
    '毛泽东思想-公共',
    '体育-公共',
    '离散数学-智能科学-2023',
    '计算机系统基础-智能科学-2023',
    '人工智能导论-智能科学-2022'
    '机器人学导论-智能科学-2022'
]


目标选课

In [48]:
mul_params = search_param_handle(target_course, campus=3, year_and_term='2024-1')
courses_list =  course_fetcher.get_courses_from_mul_params(mul_params)
print(courses_list)

[[Course(selection_number='07001044', course_name='[009101251913]围棋与中国文化', campus='西海岸校区', teaching_method='讲授', instructor='齐祥明', start_week='1-17', credits=2.0, total_hours=32, limit=40, selected_count=36, confirmed_count=0, class_time='[周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 二 (3-4节) 每周]', class_location='南402/401', syllabus='', teaching_calendar='', notes='')], [Course(selection_number='16001079', course_name='[008401101063]概率统计', campus='西海岸校区', teaching_method='讲授', instructor='王雯', start_week='1-17', credits=4.0, total_hours=64, limit=120, selected_count=39, confirmed_count=0, class_time='[周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 三 (1-2节) 每周, 周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 一 (5-6节) 每周]', class_location='南506/505', syllabus='', teaching_calendar='', notes=''), Course(selection_number='16001080', course_name='[008401101063]概率统计', campus='西海岸校区', teaching_method='讲授', instructor='王雯', start

In [53]:
importance_course = [
    '离散数学',
    '计算机系统基础',
    '人工智能导论'
    '机器人学导论'
]

unnecessary_course = [
    '篮球',
    '足球',
    '跆拳道',
]

def is_in_course(target, course_list):
    for temp_course in course_list:
        if temp_course in target:
            return True

def limit_function(course):
    
    if is_in_course(course.course_name, importance_course):
        return False
    
    # 不要的课
    if is_in_course(course.course_name, unnecessary_course):
        return True
    
    # 超人数 不选
    if float(course.limit) < int(course.selected_count) * 1:
        return True
    
    # 早八不选
    for time in course.get_class_time_list():
        if time.start_period == 1:
            return False

限制函数

In [54]:
table = CourseTable.calculate_course(courses_list, limit_function)
table.print_courses()

围棋与中国文化
07001044 [周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 二 (3-4节) 每周]

概率统计
16001079 [周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 三 (1-2节) 每周, 周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 一 (5-6节) 每周]

形势与政策-2024秋
19000176 [周次: [7, 8, 9, 10] 星期: 日 (1-2节) 每周]

毛泽东思想和中国特色社会主义理论体系概论
19000017 [周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 三 (5-6节) 双周, 周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 三 (5-7节) 单周]

体育(初级网球)
18003238 [周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 二 (5-6节) 每周]

离散数学Ⅱ
02003004 [周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 一 (3-4节) 每周, 周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 二 (1-2节) 每周]

计算机系统基础
02003003 [周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 一 (7-9节) 每周, 周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 四 (1-2节) 每周]

大学生职业发展教育

获得课程表

In [23]:
common_params = FetcherParams()
common_params.set_keyword('计算机系统基础')
common_params.sel_nj = 2023

result: list[Course] = course_fetcher.get_courses_by_params(common_params)

for course in result:
    print(course.get_class_time_list())

[周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 一 (7-9节) 每周, 周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 四 (1-2节) 每周]
[周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 一 (7-9节) 每周, 周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 四 (1-2节) 每周]
[周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 一 (7-9节) 每周, 周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 二 (5-6节) 单周, 周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 二 (5-6节) 双周]
[周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 一 (7-9节) 每周, 周次: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17] 星期: 二 (5-6节) 每周]
